In [31]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import random as rand
%matplotlib inline
import matplotlib.pyplot as plt 

import wandb

In [32]:
pip install wandb

In [33]:
sweep_config = {
    'method': 'grid', #grid, random
    'metric': {
      'name': 'loss',
      'goal': 'minimize'   
    },
    'epochs': {
            'values': [30, 50, 100]
        },
        'batch_size': {
            'values': [50,100,150]
        },
    'parameters': {
        
        'learning_rate': {
            'values': [1e-1,1e-2, 1e-3, 1e-4]
        },
        
        'optimizer': {
            'values': ['adam', 'sgd', 'adagrad', 'adadelta', 'rmsprop']
        },
    }
}

In [34]:
sweep_id = wandb.sweep(sweep_config, project="CUDA_Sheet2_Ruben_Rohil")

Create sweep with ID: qauafjpk
Sweep URL: https://wandb.ai/rohilrao/CUDA_Sheet2_Ruben_Rohil/sweeps/qauafjpk


In [35]:
def build_dataset(batch_size):
   trainset = dsets.CIFAR10('./data', train=True, download=True, transform=transforms.ToTensor())
   testset = dsets.CIFAR10('./data', train=False, download=True, transform=transforms.ToTensor())

   train_loader = torch.utils.data.DataLoader(dataset=trainset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
 
   test_loader = torch.utils.data.DataLoader(dataset=testset, 
                                          batch_size=batch_size, 
                                          shuffle=False)  

   return train_loader,test_loader

In [36]:
# DEFINE MODEL AND FORWARD PROPAGATION

class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear1 = nn.Linear(input_dim, 512)
        self.linear2 = nn.Linear(512, 128)
        self.linear3 = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()
     
    def forward(self, x):
        out = self.relu(self.linear1(x))
        out = self.relu(self.linear2(out))
        out = self.linear3(out)
        return out

In [37]:

def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'epochs': 5,
        'batch_size': 100,
        'learning_rate': 1e-3,
        'optimizer': 'sgd',
    }
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config


    
    # Define the model architecture - CIFAR10
    input_dim  = 3*32*32
    output_dim = 10
    model = LogisticRegressionModel(input_dim, output_dim)

    # Define the loss
    criterion = nn.CrossEntropyLoss().to(device) 

    #Build Dataset for CIFAR10
    train_loader, test_loader = build_dataset(config.batch_size)
  

    # Define the optimizer
    if config.optimizer=='sgd':
      optimizer = torch.optim.SGD(model.parameters(),lr=config.learning_rate, momentum=0.9)
    elif config.optimizer=='adam':
      optimizer = torch.optim.Adam(model.parameters(),lr=config.learning_rate)
    elif config.optimizer=='rmsprop':
      optimizer = torch.optim.RMSprop(model.parameters(), lr=config.learning_rate)

    model.train()
    model.to(device)

    for i in range(config.epochs):
      closs= 0
      for batch_idx, (data, target) in enumerate(train_loader):
          
          data = data.view(-1, 32*32*3)

          data, target = data.to(device), target.to(device)
          optimizer.zero_grad()
          output = model(data)
          loss = criterion(output, target)
          loss.backward()
          closs = closs + loss.item()
          optimizer.step()
          wandb.log({"batch loss":loss.item()})
      wandb.log({"loss":closs/config.batch_size}) 

In [38]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 0ozay7m5 with config:
wandb: 	learning_rate: 0.1
wandb: 	optimizer: adam


Files already downloaded and verified
Files already downloaded and verified


batch loss,2.31064
_step,2504
_runtime,39
_timestamp,1606599659
loss,11.54571


batch loss,█▅▄▆▄▄▄▃▄▄▁▆▄▄▆▆▅▄▆▄▆▆▅▃▃▄▆▃▄▄▇▆▅▄▆▄▃▅▅▃
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▁▁▁▁


wandb: Agent Starting Run: 96fda8u5 with config:
wandb: 	learning_rate: 0.1
wandb: 	optimizer: sgd


...

Run x90eme92 errored: UnboundLocalError("local variable 'optimizer' referenced before assignment",)
wandb: ERROR Run x90eme92 errored: UnboundLocalError("local variable 'optimizer' referenced before assignment",)
wandb: Agent Starting Run: 3gjpuxba with config:
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: rmsprop


Files already downloaded and verified
Files already downloaded and verified


batch loss,1.54472
_step,2504
_runtime,42
_timestamp,1606600190
loss,8.16367


batch loss,█▇▅▆▆▆▆▄▆▆▅▅▄▅▅▄▄▃▄▃▄▄▄▃▄▃▃▄▄▂▄▄▃▄▁▃▁▃▂▄
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▃▂▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# New Section